In [2]:
import numpy as np
from scipy.special import jv
import copy
from hamiltonian import hamiltonian_c, setup_scaled_H
from numba import jit, autojit

In [35]:
def find_nmax(tot,m):
    first = np.mod(tot - abs(m),2)
    return (tot-abs(m)-first)/2+1


def dbesj(x, alpha, n):
    """sequence of bessel functions where n is the number of elements of a
    bessel function of J_(alpha + k -1)(x) where k = 1...n
    """
    bessel_j = np.zeros(n+1)
    i = 0
    for k in range(1,n+1):
        bessel_j[i] = jv(alpha + k -1, x)
        i = i + 1
    y = len([num for num in bessel_j if abs(num) < 1e-20])
    return bessel_j, y

def fast_dbesj(x,alpha,n):
    k = np.arange(0,n+1,1) + alpha
    bessel_j = jv(k,x)
    y = len([num for num in bessel_j if np.abs(num) < 1e-20])
    return bessel_j,y


def chebyshev_propagator(time_step, psi, n_tot, e, d):
    """propogate function with Chebyshev Propgation"""
    epsilon = 1e-15
    #estimate upper bound with asymptotic form
    chebyshev_order = int(time_step) + 5
    Y = 0.5 * np.e* time_step
    X = (Y/chebyshev_order)**chebyshev_order/np.sqrt(2*np.pi*chebyshev_order)
    while X > epsilon:
        chebyshev_order = chebyshev_order + 10
        X = (Y/chebyshev_order)**chebyshev_order / np.sqrt(2*np.pi*chebyshev_order)
    #now compute the Bessel function
    bessel_j, nz = dbesj(time_step,0,chebyshev_order+1)
    #get the order of the polynomial, find index of first element less than epsilon
    order = next(i[0] for i in enumerate(bessel_j) if abs(i[1]) < epsilon)+1
    
    #print('Number of Chebyshev polynomials', order)
    
    psi_minus1 = psi #make sure to copy over
    psi_0 = hamiltonian_c(n_tot,psi_minus1,e,d)
    phase = np.complex(0,-1)
    cx = 2 * bessel_j[1] * phase
    psi = np.multiply(psi,np.complex(bessel_j[0],0))
    psi = np.add(np.multiply(cx,psi_0),psi)
    for i in range(2,order):
        phase = phase * np.complex(0,-1)
        cx = 2 * bessel_j[i] * phase
        psi_plus1 = hamiltonian_c(n_tot,psi_0,e,d)
        psi_plus1 = np.multiply(psi_plus1, np.complex(2,0))
        psi_plus1 = np.subtract(psi_plus1, psi_minus1)
        psi = np.add(psi, np.multiply(cx, psi_plus1))
        psi_minus1 = psi_0
        psi_0 = psi_plus1
        
    return psi

@autojit
def new_chebyshev_propagator(time_step, psi, n_tot, e, d):
    """propogate function with Chebyshev Propgation"""
    epsilon = 1e-15
    #estimate upper bound with asymptotic form
    chebyshev_order = int(time_step) + 5
    Y = 0.5 * np.e* time_step
    X = (Y/chebyshev_order)**chebyshev_order/np.sqrt(2*np.pi*chebyshev_order)
    while X > epsilon:
        chebyshev_order = chebyshev_order + 10
        X = (Y/chebyshev_order)**chebyshev_order / np.sqrt(2*np.pi*chebyshev_order)
    
    #now compute the Bessel function
    bessel_j, nz = dbesj(time_step,0,chebyshev_order+1)
    #get the order of the polynomial, find index of first element less than epsilon
    for i in range(chebyshev_order-1,0,-1):
        if bessel_j[i] >= epsilon:
            break
    order = i + 2
    #print('Number of Chebyshev polynomials', order)
    
    return evolve_chebyshev(psi,order,bessel_j,n_tot,e,d)

@autojit        
def evolve_chebyshev(psi,order,bessel_j,n_tot,e,d):
    psi_minus1 = psi 
    psi_0 = hamiltonian_c(n_tot,psi_minus1,e,d)
    phase = np.complex(0,-1)
    cx = 2 * bessel_j[1] * phase
    psi = np.multiply(psi,np.complex(bessel_j[0],0))
    psi = np.add(np.multiply(cx,psi_0),psi)
    for i in range(2,order):
        phase = phase * np.complex(0,-1)
        cx = 2 * bessel_j[i] * phase
        psi_plus1 = hamiltonian_c(n_tot,psi_0,e,d)
        psi_plus1 = np.multiply(psi_plus1, np.complex(2,0))
        psi_plus1 = np.subtract(psi_plus1, psi_minus1)
        psi = np.add(psi, np.multiply(cx, psi_plus1))
        psi_minus1 = psi_0
        psi_0 = psi_plus1
    
    return psi



In [18]:
b_field = 0.0           #BField
n_tot = 40000            #TotalAtomNumber
m = -4                #Magnetization
mag_range = 7         #MagRange
atom_range = 40       #AtomRange
spinor_phase = 0.0      #SpinorPhase
n_0 = 39996              #N_0 numbers tarting in m=0
c = 24             #C_init in Hz
q = -2.5
dt = .04/30

In [19]:
n_max = find_nmax(n_tot,m)
in_w = np.zeros(n_max, dtype = complex)
for i in range(len(in_w)):
    in_w[i] = np.complex(np.random.rand(),np.random.rand()) 
e_min,e_max,d,e, first_n0 =setup_scaled_H(q,c, n_tot, m,n_max)
scaled_dt = 2*np.pi * (e_max - e_min)*dt/2

In [20]:
%timeit chebyshev_propagator(scaled_dt,in_w,n_max,e,d)

1 loops, best of 3: 1.76 s per loop


In [21]:
%timeit new_chebyshev_propagator(scaled_dt,in_w,n_max,e,d)

1 loops, best of 3: 1.65 s per loop


In [565]:
a1 = chebyshev_propagator(scaled_dt,in_w,n_max,e,d)
a2 = new_chebyshev_propagator(scaled_dt,in_w,n_max,e,d)

print(all(a1==a2))
print(a1[:10])
print(a2[:10])

True
[-0.09815406+1.13838915j  0.05846620+1.0368539j   0.23125683+0.36787668j
  0.97670759-0.02730613j  0.36435184-0.26244547j  0.28054540-0.78344714j
 -0.02982128-0.02989204j -0.49953133+0.40334915j  0.30762180+0.87939251j
  0.73447176+0.63592759j]
[-0.09815406+1.13838915j  0.05846620+1.0368539j   0.23125683+0.36787668j
  0.97670759-0.02730613j  0.36435184-0.26244547j  0.28054540-0.78344714j
 -0.02982128-0.02989204j -0.49953133+0.40334915j  0.30762180+0.87939251j
  0.73447176+0.63592759j]


In [511]:
@autojit
def func2(x,y):
    ans =  np.subtract(x,y)
   
    return ans
    
@autojit
def func3(x,y,z):
    l = len(x)
    for i in range(l):
        z[i] = x[i] - y[i]
  
    return z

nn = 10

in_w2 = np.zeros(nn, dtype = complex)
zz = np.zeros(nn, dtype = complex)
for i in range(nn):
    temp = np.complex(np.random.rand(),np.random.rand())

    in_w2[i] = temp
ph = np.complex(2,2)

In [512]:
%timeit func2(in_w2,in_w2)

The slowest run took 73519.11 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 235 ns per loop


In [513]:
%timeit func3(in_w2,in_w2,zz)

The slowest run took 123310.56 times longer than the fastest. This could mean that an intermediate result is being cached 
1000000 loops, best of 3: 264 ns per loop


In [514]:
ans2 = func2(in_w2,in_w2)
ans3 = func3(in_w2, in_w2, zz)

In [515]:
print(all(ans2==ans3))

True


In [26]:
%timeit dbesj(.1,0,10)

10000 loops, best of 3: 64 µs per loop


In [34]:
%timeit fast_dbesj(.1,0,10)

10000 loops, best of 3: 45.5 µs per loop


In [29]:
dbesj(.1,0,3) 

(array([ 0.99750156,  0.04993753,  0.00124896,  0.        ]), 1)

In [36]:
fast_dbesj(.1,0,3)

NotImplementedError: offset=53 opcode=0x84 opname=MAKE_FUNCTION